In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
tanscripts_read= "/home/andre/Documents/trabalho/datasets/Disease_genes/mane/Homo_sapiens.GRCh38.112.chr.gff3"
tanscripts_df = pd.read_csv(tanscripts_read, sep="\t", header=None)
#tanscripts_df

In [3]:
tanscripts_df = tanscripts_df.rename(columns={0: 'Chr', 1: 'Genome', 2: 'Type', 3: 'First_index', 4: 'Last_index', 5: 'Value', 6: 'Signal', 7: 'Unknown', 8: 'Identifiers'})
tanscripts_df

,Chr,Genome,Type,First_index,Last_index,Value,Signal,Unknown,Identifiers
0,1,GRCh38,chromosome,1.0,248956422.0,.,.,.,"ID=chromosome:1;Alias=CM000663.2,chr1,NC_00000..."
1,1,.,biological_region,10469.0,11240.0,1.3e+03,.,.,external_name=oe %3D 0.79;logic_name=cpg
2,1,.,biological_region,10650.0,10657.0,0.999,+,.,logic_name=eponine
3,1,.,biological_region,10655.0,10657.0,0.999,-,.,logic_name=eponine
4,1,.,biological_region,10678.0,10687.0,0.999,+,.,logic_name=eponine
...,...,...,...,...,...,...,...,...,...
3516515,Y,.,biological_region,57215804.0,57215813.0,0.999,-,.,logic_name=eponine
3516516,Y,.,biological_region,57215832.0,57215840.0,0.999,+,.,logic_name=eponine
3516517,Y,.,biological_region,57215833.0,57215842.0,0.999,-,.,logic_name=eponine
3516518,Y,.,biological_region,57215862.0,57215867.0,0.999,+,.,logic_name=eponine


In [4]:
tanscripts_df['Type'].unique()

array(['chromosome', 'biological_region', 'ncRNA_gene', 'lnc_RNA', 'exon',
       nan, 'pseudogene', 'pseudogenic_transcript', 'miRNA', 'gene',
       'mRNA', 'five_prime_UTR', 'CDS', 'three_prime_UTR', 'snRNA',
       'transcript', 'unconfirmed_transcript', 'snoRNA', 'scRNA', 'rRNA',
       'processed_transcript', 'V_gene_segment', 'D_gene_segment',
       'J_gene_segment', 'C_gene_segment', 'tRNA'], dtype=object)

In [5]:
tanscripts_df["Identifiers"][0]

'ID=chromosome:1;Alias=CM000663.2,chr1,NC_000001.11'

In [ ]:
#chromosmes = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
#genes = tanscripts_df.loc[tanscripts_df['Type'] == "gene"]
#CDSs = tanscripts_df.loc[tanscripts_df['Type'] == "CDS"]
#UTR3 = tanscripts_df.loc[tanscripts_df['Type'] == "three_prime_UTR"]
#UTR5 = tanscripts_df.loc[tanscripts_df['Type'] == "five_prime_UTR"]
#exons = tanscripts_df.loc[tanscripts_df['Type'] == "exon"]

In [ ]:
#UTR3["Identifiers"]

In [12]:
#UTR5_overlap = UTR5.loc[UTR5["Identifiers"] == UTR5["Identifiers"][3516301]]

In [ ]:
#UTR5_overlap.columns.tolist()

In [6]:
def transcripts_NoOverlaps(element_type = "CDS"):
    element_df = tanscripts_df.loc[tanscripts_df['Type'] == element_type]
    elements_dict = {}
    #elements_dict.update({"header": element_df.columns.tolist()})
    count = 0
    for row in element_df.index:
        identifiers = element_df["Identifiers"][row]
        transcript_id = identifiers.split(";")[0]
        transcript_length = element_df["Last_index"][row] - element_df["First_index"][row]
        if transcript_id not in elements_dict.keys():
        #if elements_dict.get(transcript_id) == None:
            #elements_dict.update({transcript_id: (element_df.values[row], transcript_length)})
            elements_dict.update({transcript_id: (element_df.iloc[[count]], transcript_length)})
        else:
            #print("resolving overlap...")
            if transcript_length > elements_dict[transcript_id][1]:
                #elements_dict.update({transcript_id: (element_df.values[row], transcript_length)})
                elements_dict.update({transcript_id: (element_df.iloc[[count]], transcript_length)})
        count+=1

    #transcripts_matrix = []
    #for value in elements_dict.values():
    #    transcripts_matrix.append(value[0])
    return elements_dict
    #return pd.DataFrame(elements_dict.values())
    #return pd.DataFrame(transcripts_matrix)

In [7]:
def transcripts_dict_toDF(dictionary):
    transcripts_matrix = []
    concat_list = []
    for value in dictionary.values():
        concat_list.append(value[0])
    return pd.concat(concat_list)

In [ ]:
#fivePrime_UTR_dict = transcripts_NoOverlaps("five_prime_UTR")

In [ ]:
#for value in fivePrime_UTR_dict.values():
#    print(type(value[0]))
#    break

In [ ]:
#UTR5_noOverlaps_df = transcripts_dict_toDF(fivePrime_UTR_dict)

In [ ]:
#UTR5_noOverlaps_df

In [ ]:
#UTR5_noOverlaps_df["Identifiers"][0]

In [8]:
def element_matrix(chromosomes_df, sequence_df):
    """
    The matrix representing all intances of a sequence type available from the original transcripts dataframe
    sequence_df = Genes, CDSs, 5 or 3 prime UTR, Exons
    """
    Chromosomes_matrix = []
    for line in chromosomes_df.index:
        chromosome = chromosomes_df["Chr"][line]
        print("searching citobands in chromossome: ", chromosome)
        chr_size = int(chromosomes_df["Last_index"][line])
        chr_sequence = sequence_df.loc[sequence_df['Chr'] == chromosome]
        chr_array = np.zeros((chr_size), dtype=int)
        #elements region
        for row in chr_sequence.index:
            chr_array[int(chr_sequence["First_index"][row]) : int(chr_sequence["Last_index"][row])] += 1
                
        
        Chromosomes_matrix.append(chr_array)
    return Chromosomes_matrix

In [9]:
def calc_elementDensity(cytobands_df, region_matrix, col_name="Element_density"):
    #cytobands_df = cytobands_df.drop(cytobands_df[cytobands_df['Chromossome'] == 'chrM'].index)
    densities = []
    chromosomes = cytobands_df["Chromossome"].unique()
    for chrom in range(len(cytobands_df["Chromossome"].unique())):             
        chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == cytobands_df["Chromossome"].unique()[chrom]]
        #Calculo da densidade para cada cytoband
        for row in chr_cytobands.index:
            cytoband_array = region_matrix[chrom][int(chr_cytobands["First_index"][row]):int(chr_cytobands["Last_index"][row])]
            dr_length = np.count_nonzero(cytoband_array)
            cytoband_length = chr_cytobands["Size"][row]
            density = dr_length/cytoband_length
            densities.append(round(density, 10))
        
    cytobands_df.insert(len(cytobands_df.columns), col_name, densities, True)

In [21]:
def calc_ExonIntronDensity(cytobands_df, region_matrix, exons_matrix, col_name1="exon_density", col_name2="intron_density"):
    #cytobands_df = cytobands_df.drop(cytobands_df[cytobands_df['Chromossome'] == 'chrM'].index)
    intron_densities = []
    exon_densities = []
    chromosomes = cytobands_df["Chromossome"].unique()
    for chrom in range(len(cytobands_df["Chromossome"].unique())):
        print("Calculating densities in chromosome: ", cytobands_df["Chromossome"].unique()[chrom], " ...")
        chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == cytobands_df["Chromossome"].unique()[chrom]]
        #Calculo da densidade para cada cytoband
        for row in chr_cytobands.index:
            ############################ Resolver problema, o conjunto dos exons contem todo o conjunto dos CDSs ###############################
            region_cytoband_array = region_matrix[chrom][int(chr_cytobands["First_index"][row]):int(chr_cytobands["Last_index"][row])]
            exon_cytoband_array = exons_matrix[chrom][int(chr_cytobands["First_index"][row]):int(chr_cytobands["Last_index"][row])]
            region_indexes = np.where(region_cytoband_array != 0)[0].tolist()
            exon_indexes = np.where(exon_cytoband_array != 0)[0].tolist()
            print("exons in CDS region: ", np.intersect1d(region_indexes, exon_indexes))
            print("introns in CDS region: ", np.setdiff1d(region_indexes, exon_indexes))
            exon_length = len(np.intersect1d(region_indexes, exon_indexes))    #indices que estam na região e são exões
            intron_length = len(np.setdiff1d(region_indexes, exon_indexes))      #indices que estam na região mas não são exões
            #for region_element in region_indexes:
            #    if region_element in exon_indexes:
            #        exon_length+=1
            #    elif region_element not in exon_indexes:
            #        intron_length+=1
            cytoband_length = chr_cytobands["Size"][row]
            exon_density = exon_length/cytoband_length
            intron_density = intron_length/cytoband_length
            exon_densities.append(round(exon_density, 10))
            intron_densities.append(round(intron_density, 10))   #max 6
            
        
    cytobands_df.insert(len(cytobands_df.columns), col_name1, exon_densities, True)
    cytobands_df.insert(len(cytobands_df.columns), col_name2, intron_densities, True)

In [18]:
array1 = [1, 2, 3, 6, 7, 8]
array2 = [2, 3, 4, 7, 8, 9]
print(np.intersect1d(array1, array2))
print(np.setdiff1d(array1, array2))

[2 3 7 8]
[1 6]


In [11]:
def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    df_cols = ""
    for col in df.columns[:-1]:
        df_cols += col + "\t"
    df_cols += df.columns.values[-1]+ "\n"
    f.write(str(df_cols))
    for row in df.values:
        df_row = ""
        for col in row[:-1]:
            df_row += str(col) + "\t"
        df_row += str(row[-1])+ "\n"
        f.write(df_row)
    f.close

In [12]:
cytoPath_read = "/home/andre/Documents/trabalho/citobands/cytobandFiltered_processed_str_dr.txt"
cytoPath_write = "/home/andre/Documents/trabalho/citobands/cytobandFiltered_processed_str_dr_dg.txt"

cytobands_df = pd.read_csv(cytoPath_read, sep="\t")#, header=None))
cytobands_df2 = pd.read_csv(cytoPath_write, sep="\t")

cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404
...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002


In [13]:
chromosmes = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
UTR5 = tanscripts_df.loc[tanscripts_df['Type'] == "five_prime_UTR"]
exons = tanscripts_df.loc[tanscripts_df['Type'] == "exon"]

In [14]:
UTR5_matrix = element_matrix(chromosmes, UTR5)

searching citobands in chromossome:  1
searching citobands in chromossome:  10
searching citobands in chromossome:  11
searching citobands in chromossome:  12
searching citobands in chromossome:  13
searching citobands in chromossome:  14
searching citobands in chromossome:  15
searching citobands in chromossome:  16
searching citobands in chromossome:  17
searching citobands in chromossome:  18
searching citobands in chromossome:  19
searching citobands in chromossome:  2
searching citobands in chromossome:  20
searching citobands in chromossome:  21
searching citobands in chromossome:  22
searching citobands in chromossome:  3
searching citobands in chromossome:  4
searching citobands in chromossome:  5
searching citobands in chromossome:  6
searching citobands in chromossome:  7
searching citobands in chromossome:  8
searching citobands in chromossome:  9
searching citobands in chromossome:  MT
searching citobands in chromossome:  X
searching citobands in chromossome:  Y


In [15]:
calc_elementDensity(cytobands_df, UTR5_matrix, "5P UTR_density")
cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,5P UTR_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.012022
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.003561
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.006496
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.004600
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.005499
...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.001027
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.000346
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.000675
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.000838


In [16]:
exon_matrix = element_matrix(chromosmes, exons)

searching citobands in chromossome:  1
searching citobands in chromossome:  10
searching citobands in chromossome:  11
searching citobands in chromossome:  12
searching citobands in chromossome:  13
searching citobands in chromossome:  14
searching citobands in chromossome:  15
searching citobands in chromossome:  16
searching citobands in chromossome:  17
searching citobands in chromossome:  18
searching citobands in chromossome:  19
searching citobands in chromossome:  2
searching citobands in chromossome:  20
searching citobands in chromossome:  21
searching citobands in chromossome:  22
searching citobands in chromossome:  3
searching citobands in chromossome:  4
searching citobands in chromossome:  5
searching citobands in chromossome:  6
searching citobands in chromossome:  7
searching citobands in chromossome:  8
searching citobands in chromossome:  9
searching citobands in chromossome:  MT
searching citobands in chromossome:  X
searching citobands in chromossome:  Y


In [20]:
calc_ExonIntronDensity(cytobands_df, UTR5_matrix, exon_matrix, col_name1="UTR5_exon_density", col_name2="UTR5_intron_density")
cytobands_df

Calculating densities in chromosome:  chr1  ...
exons in CDS region:  [  65419   65420   65421 ... 2228763 2228764 2228765]
introns in CDS region:  []
exons in CDS region:  [  91534   91535   91536 ... 2355422 2355423 2355424]
introns in CDS region:  []
exons in CDS region:  [ 686290  686291  686292 ... 1485547 1485548 1485549]
introns in CDS region:  []
exons in CDS region:  [ 569759  569760  569761 ... 1988475 1988476 1988477]
introns in CDS region:  []
exons in CDS region:  [  29028   29029   29030 ... 3397628 3397629 3397630]
introns in CDS region:  []
exons in CDS region:  [   6853    6854    6855 ... 3372933 3372934 3372935]
introns in CDS region:  []
exons in CDS region:  [  47061   47062   47063 ... 4184827 4184828 4184829]
introns in CDS region:  []
exons in CDS region:  [  19499   19500   19501 ... 3459498 3459499 3459500]
introns in CDS region:  []
exons in CDS region:  [  91742   91743   91744 ... 3958880 3958881 3958882]
introns in CDS region:  []
exons in CDS region:  [  

KeyboardInterrupt: 

# Code running

In [11]:
cytobands_df2

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,5P_UTR_density,UTR5Intron_density,UTR5Exon_density,3P_UTR_density,UTR3Intron_density,UTR3Exon_density,Exons_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.038,0.022,0.015,0.033,0.022,0.021,0.012,0.025,0.019,0.171
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.016,0.011,0.005,0.022,0.010,0.014,0.004,0.013,0.006,0.079
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.024,0.010,0.008,0.034,0.009,0.020,0.006,0.012,0.010,0.082
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.013,0.004,0.002,0.018,0.003,0.010,0.005,0.004,0.004,0.047
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.030,0.010,0.008,0.036,0.010,0.019,0.005,0.014,0.010,0.093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.004,0.001,0.001,0.003,0.001,0.003,0.001,0.001,0.003,0.016
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002,0.004,0.004,0.001,0.004,0.003,0.000,0.004,0.002,0.045
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.005,0.004,0.002,0.004,0.004,0.002,0.001,0.005,0.002,0.036
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.003,0.002,0.000,0.002,0.003,0.001,0.001,0.003,0.001,0.034


In [24]:
def study_elementDens(cytoband_df, element_type = "CDS"):
    chromosmes = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
    element_dict = transcripts_NoOverlaps(element_type)
    element_noOverlaps_df = transcripts_dict_toDF(element_dict)
    transcript_matrix = element_matrix(chromosmes, element_noOverlaps_df)
    colname = element_type+"s_density"
    calc_elementDensity(cytoband_df, transcript_matrix, colname)
    writeNewtxt(cytoPath_write, cytoband_df)

In [37]:
def study_intronExonsDens(cytoband_df, element_type = "CDS"):
    chromosmes = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
    element_dict = transcripts_NoOverlaps(element_type)
    element_noOverlaps_df = transcripts_dict_toDF(element_dict)
    exons_dict = transcripts_NoOverlaps("exon")
    exons_noOverlaps_df = transcripts_dict_toDF(exons_dict)
    print("a fazer a matriz...")
    element_introns_exons_matrix = introns_exons_matrix(chromosmes, element_noOverlaps_df, exons_noOverlaps_df)
    colname1 = element_type+"Intron_density"
    colname2 = element_type+"Exon_density"
    print("a fazer os calculos...")
    calc_ExonIntronDensity(cytoband_df, element_introns_exons_matrix, colname1, colname2)
    writeNewtxt(cytoPath_write, cytobands_df)

## CDS density study

In [31]:
study_elementDens(cytobands_df, "CDS")

searching citobands in chromossome:  1
searching citobands in chromossome:  10
searching citobands in chromossome:  11
searching citobands in chromossome:  12
searching citobands in chromossome:  13
searching citobands in chromossome:  14
searching citobands in chromossome:  15
searching citobands in chromossome:  16
searching citobands in chromossome:  17
searching citobands in chromossome:  18
searching citobands in chromossome:  19
searching citobands in chromossome:  2
searching citobands in chromossome:  20
searching citobands in chromossome:  21
searching citobands in chromossome:  22
searching citobands in chromossome:  3
searching citobands in chromossome:  4
searching citobands in chromossome:  5
searching citobands in chromossome:  6
searching citobands in chromossome:  7
searching citobands in chromossome:  8
searching citobands in chromossome:  9
searching citobands in chromossome:  MT
searching citobands in chromossome:  X
searching citobands in chromossome:  Y


NameError: name 'writeNewtxt' is not defined

In [33]:
cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.016467
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.006066
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.010266
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.005996
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.009799
...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.000917
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002263
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.001096
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.001369


In [22]:
#CDSs_matrix = element_matrix(chromosmes, CDSs)

In [23]:
#calc_elementDensity(cytobands_df, CDSs_matrix, "CDSs_density")
#cytobands_df

In [24]:
#writeNewtxt(cytoPath_write, cytobands_df)

## Exon and Intron densities within CDSs (Coding Sequences)

In [35]:
study_intronExonsDens(cytobands_df, "CDS")

searching citobands in chromossome:  1
searching citobands in chromossome:  10
searching citobands in chromossome:  11
searching citobands in chromossome:  12
searching citobands in chromossome:  13
searching citobands in chromossome:  14
searching citobands in chromossome:  15
searching citobands in chromossome:  16
searching citobands in chromossome:  17
searching citobands in chromossome:  18
searching citobands in chromossome:  19
searching citobands in chromossome:  2
searching citobands in chromossome:  20
searching citobands in chromossome:  21
searching citobands in chromossome:  22
searching citobands in chromossome:  3
searching citobands in chromossome:  4
searching citobands in chromossome:  5
searching citobands in chromossome:  6
searching citobands in chromossome:  7
searching citobands in chromossome:  8
searching citobands in chromossome:  9
searching citobands in chromossome:  MT
searching citobands in chromossome:  X
searching citobands in chromossome:  Y


In [36]:
cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.016467,0.020383,0.013959
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.006066,0.010302,0.002620
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.010266,0.010090,0.005856
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.005996,0.003048,0.001599
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.009799,0.010669,0.004489
...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.000917,0.001011,0.000607
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002263,0.004221,0.002790
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.001096,0.002814,0.000916
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.001369,0.001053,0.000025


In [25]:
#CDS_introns_exons_matrix = introns_exons_matrix(chromosmes, CDSs, exons)

In [26]:
#calc_ExonIntronDensity(cytobands_df, CDS_introns_exons_matrix, "CDSIntron_density", "CDSExon_density")
#cytobands_df

In [27]:
#writeNewtxt(cytoPath_write, cytobands_df)

## Prime 5 UTR density study

In [28]:
#UTR5_matrix = element_matrix(chromosmes, UTR5)

In [29]:
#calc_elementDensity(cytobands_df, UTR5_matrix, "5P_UTR_density")
#cytobands_df

In [30]:
#writeNewtxt(cytoPath_write, cytobands_df)

## Exon and Intron densities within 5 Prime UTR

In [31]:
#UTR5_introns_exons_matrix = introns_exons_matrix(chromosmes, UTR5, exons)

In [32]:
#calc_ExonIntronDensity(cytobands_df2, UTR5_introns_exons_matrix, "UTR5Intron_density", "UTR5Exon_density")
#cytobands_df
#cytobands_df2

In [33]:
#writeNewtxt(cytoPath_write, cytobands_df2)

## Prime 3 UTR density study

In [34]:
#UTR3_matrix = element_matrix(chromosmes, UTR3)

In [35]:
#calc_elementDensity(cytobands_df2, UTR3_matrix, "3P_UTR_density")
#cytobands_df2

In [36]:
#writeNewtxt(cytoPath_write, cytobands_df2)

## Exon and Intron densities within 3 Prime UTR

In [37]:
#UTR3_introns_exons_matrix = introns_exons_matrix(chromosmes, UTR3, exons)

In [38]:
#calc_ExonIntronDensity(cytobands_df2, UTR3_introns_exons_matrix, "UTR3Intron_density", "UTR3Exon_density")
#cytobands_df2

In [39]:
#writeNewtxt(cytoPath_write, cytobands_df2)

## Exons density study

In [44]:
#exons_matrix = element_matrix(chromosmes, exons)

searching citobands in chromossome:  1
searching citobands in chromossome:  10
searching citobands in chromossome:  11
searching citobands in chromossome:  12
searching citobands in chromossome:  13
searching citobands in chromossome:  14
searching citobands in chromossome:  15
searching citobands in chromossome:  16
searching citobands in chromossome:  17
searching citobands in chromossome:  18
searching citobands in chromossome:  19
searching citobands in chromossome:  2
searching citobands in chromossome:  20
searching citobands in chromossome:  21
searching citobands in chromossome:  22
searching citobands in chromossome:  3
searching citobands in chromossome:  4
searching citobands in chromossome:  5
searching citobands in chromossome:  6
searching citobands in chromossome:  7
searching citobands in chromossome:  8
searching citobands in chromossome:  9
searching citobands in chromossome:  MT
searching citobands in chromossome:  X
searching citobands in chromossome:  Y


In [46]:
#calc_elementDensity(cytobands_df2, exons_matrix, "Exons_density")
#cytobands_df2

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,5P_UTR_density,UTR5Intron_density,UTR5Exon_density,3P_UTR_density,UTR3Intron_density,UTR3Exon_density,Exons_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.038,0.022,0.015,0.033,0.022,0.021,0.012,0.025,0.019,0.171
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.016,0.011,0.005,0.022,0.010,0.014,0.004,0.013,0.006,0.079
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.024,0.010,0.008,0.034,0.009,0.020,0.006,0.012,0.010,0.082
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.013,0.004,0.002,0.018,0.003,0.010,0.005,0.004,0.004,0.047
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.030,0.010,0.008,0.036,0.010,0.019,0.005,0.014,0.010,0.093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.004,0.001,0.001,0.003,0.001,0.003,0.001,0.001,0.003,0.016
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002,0.004,0.004,0.001,0.004,0.003,0.000,0.004,0.002,0.045
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.005,0.004,0.002,0.004,0.004,0.002,0.001,0.005,0.002,0.036
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.003,0.002,0.000,0.002,0.003,0.001,0.001,0.003,0.001,0.034


In [47]:
#writeNewtxt(cytoPath_write, cytobands_df2)

In [43]:
#cytobands_df.values[cytobands_df.index[70]]

In [12]:
def sumOf_columns(df, colums=["CDSIntron_density", "UTR5Intron_density", "UTR3Intron_density"], newColName = "Introns_density"):
    """
    colums is a list of string
    """
    #cytobands_df = cytobands_df.drop(cytobands_df[cytobands_df['Chromossome'] == 'chrM'].index)
    new_densites = []
    for row in df.index:
        density =0
        for column in colums:
            density += df[column][row]
        new_densites.append(round(density, 3))
        
    df.insert(len(df.columns), newColName, new_densites, True)

In [14]:
#sumOf_columns(cytobands_df2)
cytobands_df2

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,5P_UTR_density,UTR5Intron_density,UTR5Exon_density,3P_UTR_density,UTR3Intron_density,UTR3Exon_density,Exons_density,Introns_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.038,0.022,0.015,0.033,0.022,0.021,0.012,0.025,0.019,0.171,0.069
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.016,0.011,0.005,0.022,0.010,0.014,0.004,0.013,0.006,0.079,0.034
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.024,0.010,0.008,0.034,0.009,0.020,0.006,0.012,0.010,0.082,0.031
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.013,0.004,0.002,0.018,0.003,0.010,0.005,0.004,0.004,0.047,0.011
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.030,0.010,0.008,0.036,0.010,0.019,0.005,0.014,0.010,0.093,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.004,0.001,0.001,0.003,0.001,0.003,0.001,0.001,0.003,0.016,0.003
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002,0.004,0.004,0.001,0.004,0.003,0.000,0.004,0.002,0.045,0.012
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.005,0.004,0.002,0.004,0.004,0.002,0.001,0.005,0.002,0.036,0.013
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.003,0.002,0.000,0.002,0.003,0.001,0.001,0.003,0.001,0.034,0.008


In [17]:
writeNewtxt(cytoPath_write, cytobands_df2)